In [1]:
import polars as pl
from polars.exceptions import ColumnNotFoundError
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['timezone'] = 'Europe/Moscow'

from datetime import datetime, timezone, timedelta
from zoneinfo import ZoneInfo
import pickle
import json
from tqdm.notebook import tqdm
from itertools import product
import random

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, root_mean_squared_error

from bot.core.db.postgres_manager import DBManager
from bot.config.credentials import host, user, password, db_name
db_params = {'host': host, 'user': user, 'password': password, 'dbname': db_name}
db_manager = DBManager(db_params)

from bot.analysis.backtesting import backtest
from bot.utils.pair_trading import *
from bot.analysis.strategy_analysis import analyze_strategy
from bot.utils.files import load_config



In [2]:
def run_test(df, wind, in_, out_, open_method, dist_in, dist_out, force_close):
    cols = ['time', 'ts', token_1, f'{token_1}_size', f'{token_1}_bid_price', f'{token_1}_ask_price',
            f'{token_1}_bid_size', f'{token_1}_ask_size', token_2, f'{token_2}_size',
            f'{token_2}_bid_price', f'{token_2}_ask_price', f'{token_2}_bid_size', f'{token_2}_ask_size',
            f'spread_{wind}_{tf}', f'spread_mean_{wind}_{tf}', f'spread_std_{wind}_{tf}', f'z_score_{wind}_{tf}']

    tdf = df.select(cols).rename({f'z_score_{wind}_{tf}': 'z_score', f'spread_{wind}_{tf}': 'spread',
                f'spread_mean_{wind}_{tf}': 'spread_mean', f'spread_std_{wind}_{tf}': 'spread_std'}).drop_nulls()

    trades_df = backtest(tdf, token_1, token_2, dp_1, dp_2,
            thresh_low_in=-in_, thresh_low_out=-out_, thresh_high_in=in_, thresh_high_out=out_,
            long_possible=True, short_possible=True,
            balance=100, order_size=50, qty_method=qty_method, std_1=std_1, std_2=std_2,
            fee_rate=0.001, sl_std=sl_std, sl_dist=1.0, sl_method='leave',
            sl_seconds = 60, open_method=open_method, close_method=close_method, 
            leverage=leverage, dist_in=dist_in, dist_out=dist_out, force_close=force_close,
            verbose=0)

    if trades_df.height > 1:
        metrics = analyze_strategy(trades_df, start_date=valid_time, end_date=end_date, initial_balance=100.0)
        return tdf, trades_df, metrics
    else:
        return tdf, pl.DataFrame(), dict()

In [3]:
def get_coins_info(token_1, token_2):
    dp_1 = float(coin_information['bybit_linear'][token_1 + '_USDT']['qty_step'])
    ps_1 = int(coin_information['bybit_linear'][token_1 + '_USDT']['price_scale'])
    dp_2 = float(coin_information['bybit_linear'][token_2 + '_USDT']['qty_step'])
    ps_2 = int(coin_information['bybit_linear'][token_2 + '_USDT']['price_scale'])

    return dp_1, dp_2, ps_1, ps_2

In [4]:
pairs_file = './data/token_pairs.txt'

all_pairs = pl.read_parquet('./data/pair_selection/all_pairs.parquet')

with open("./data/coin_information.pkl", "rb") as f:
    coin_information = pickle.load(f)

token_pairs = []
with open(pairs_file, 'r') as file:
    for line in file:
        a, b = line.strip().split()
        token_pairs.append((a, b))
len(token_pairs)

431

In [20]:
# Параметры
config = load_config('./bot/config/config.yaml')
valid_time = config['valid_time']
end_time = config['end_time']
force_close = False
qty_method = 'usdt_neutral'

leverage = 2
min_order_size = 42
max_order_size = 50
fee_rate = 0.001
sl_ratio = 0.1
sl_std = 5.0
close_method = 'direct'

tf = '1h'
winds = (64, 72, 96)
spr_methods = ('dist', 'lr', )
open_methods = ('direct', 'reverse_static', 'reverse_dynamic')
in_min = 1.6
in_max = 2.75
out_min = 0.0
out_max = 0.5
dist_min = 0.05
dist_max = 1.0

metrics_arr = []

In [21]:
n_iters = 10

for _ in tqdm(range(n_iters)):
    token_1, token_2 = random.choice(token_pairs)
    spr_method = random.choice(spr_methods)
    pair_stats = all_pairs.filter((pl.col('coin1') == token_1) & (pl.col('coin2') == token_2))

    if pair_stats.height > 0:
        std_1 = pair_stats['std_1'][0]
        std_2 = pair_stats['std_2'][0]
    else:
        print(token_1, token_2, 'No Stats Data!')
        continue

    filepath = f'./data/pair_backtest/{token_1}_{token_2}_1h_{spr_method}.parquet'

    try:
        df = pl.read_parquet(filepath, low_memory=True, rechunk=True, use_pyarrow=True)
        end_date = df['time'][-1]
    except FileNotFoundError:
        print(f'FileNotFoundError!: {filepath}')
        continue

    in_ = round(random.uniform(in_min, in_max), 2)
    out_ = round(random.uniform(out_min, out_max), 2)
    wind = random.choice(winds)
    open_method = random.choice(open_methods)

    dp_1, dp_2, ps_1, ps_2 = get_coins_info(token_1, token_2)

    if open_method == 'direct':
        dist_in = 0.0
        dist_out = 0.0
    elif open_method in ('reverse_static', 'reverse_dynamic'):
        dist_in = round(random.uniform(dist_min, dist_max), 2)
        dist_out = round(random.uniform(dist_min, dist_max), 2)

    tdf, trades_df, metrics = run_test(df, wind, in_, out_, open_method, dist_in, dist_out, force_close)

    tick_df, agg_df = load_data(token_1, token_2, valid_time, end_time, tf, max(winds), db_manager)

    tick_df = tick_df.with_columns((pl.col(token_1).log() - pl.col(token_2).log()).alias('log_spread'))
    agg_df = agg_df.with_columns((pl.col(token_1).log() - pl.col(token_2).log()).alias('log_spread'))
    
    start_ts = int(datetime.timestamp(valid_time))
    lr_df = create_zscore_df(token_1, token_2, tick_df, agg_df, tf, np.array([wind]), start_ts, 
                                 median_length=6, spr_method='lr')

    for trade in trades_df.to_dicts():
        open_time = trade['open_time']
        side_1 = 'long' if trade['pos_side'] == 1 else 'short'
        side_2 = 'long' if trade['pos_side'] == 2 else 'short'
    
        lr_hist = lr_df.filter(pl.col('time') < open_time).tail(wind * 12 * 60)
        lr_spread = lr_hist[f'spread_{wind}_1h'].to_numpy()
    
        params = {'tf': '1h', 'wind': wind, 'side_1': side_1, 'side_2': side_2}
        stats = get_open_time_stats(token_1, token_2, open_time, tick_df, agg_df, lr_spread, params, coin_information)
    
        trade.pop('open_time')
        trade.pop('close_time')
        pars = {'token_1': token_1, 'token_2': token_2, 'spr_method': spr_method, 'wind': wind, 
                'in_': in_, 'out_': out_, 'open_method': open_method, 'dist_in': dist_in, 'dist_out': dist_out}
        
        log = pars | trade | stats
        json_log = json.dumps(log, default=float, ensure_ascii=False)
        with open('./logs/backtest_trades.jsonl', 'a', encoding='utf-8') as f:
            f.write(json_log + '\n')
    
        metrics_arr.append(log)

  0%|          | 0/10 [00:00<?, ?it/s]

In [24]:
stats

{'std_1_180d': 0.04952307052042427,
 'std_2_180d': 0.043343864199329236,
 'beta_1_180d': 1.3149990363305595,
 'beta_2_180d': 0.7117451964249027,
 'pv_1': 0.11238165052033433,
 'pv_2': 0.119289827170518,
 'tls_beta_180d': 0.8360815592679074,
 'coint_180d': 0,
 'johansen_beta_180d': 0.7522361859382103,
 'mean_wind': -0.6095946297731787,
 'std_1_wind': 0.007149971265637838,
 'std_2_wind': 0.0071171687969130756,
 'mean_12h': -0.5847256749282445,
 'std_1_12h': 0.010900704719826819,
 'std_2_12h': 0.006461228828215143,
 'mean_diff': -4.079588898968423,
 'tls_beta_wind': np.float64(1.3711764833195277),
 'tls_beta_12h': np.float64(0.587729924198411),
 'hurst_wind': np.float64(0.5412933501634817),
 'hurst_12h': np.float64(0.537806090334077),
 'spread_rsi_5m': 56.11561630907134,
 'spread_rsi_1h': 75.60354512628865,
 'rsi_t1_5m': 49.707906091780636,
 'rsi_t2_5m': 41.78812446526753,
 'rsi_t1_1h': 40.72107297400704,
 'rsi_t2_1h': 18.587081775757042,
 'profit_sensitivity_12h_mean': np.float64(-0.7534

In [23]:
pl.DataFrame(metrics_arr).sample(5)

token_1,token_2,spr_method,wind,in_,out_,open_method,dist_in,dist_out,open_ts,close_ts,qty_1,qty_2,open_price_1,close_price_1,open_price_2,close_price_2,pos_side,fees,profit_1,profit_2,total_profit,reason,std_1_180d,std_2_180d,beta_1_180d,beta_2_180d,pv_1,pv_2,tls_beta_180d,coint_180d,johansen_beta_180d,mean_wind,std_1_wind,std_2_wind,mean_12h,std_1_12h,std_2_12h,mean_diff,tls_beta_wind,tls_beta_12h,hurst_wind,hurst_12h,spread_rsi_5m,spread_rsi_1h,rsi_t1_5m,rsi_t2_5m,rsi_t1_1h,rsi_t2_1h,profit_sensitivity_12h_mean,profit_sensitivity_12h_pos,profit_sensitivity_12h_neg,profit_sensitivity_wind_mean,profit_sensitivity_wind_pos,profit_sensitivity_wind_neg,trend_wind,trend_12h,half_life_log_spread,half_life_lr_spread,btc_corr_1,btc_corr_2,eth_corr_1,eth_corr_2,sol_corr_1,sol_corr_2
str,str,str,i64,f64,f64,str,f64,f64,i64,i64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""JASMY""","""VET""","""lr""",72,1.87,0.05,"""reverse_static""",0.74,0.61,1764319900,1764427995,13083.0,7078.0,0.007628,0.007274,0.0141,0.01339,1,0.389537,-4.826345,4.830806,0.004461,1,0.049523,0.043344,1.314999,0.711745,0.112382,0.11929,0.836082,0,0.752236,-0.596037,0.005943,0.005964,-0.603728,0.006319,0.006236,1.290312,1.036154,0.513746,0.497499,0.63944,45.002277,28.38241,71.309236,76.167132,57.074861,69.547762,0.479747,0.478865,0.480597,0.50126,0.500562,0.501933,2.17477,4.669568,6.342777,7.895789,0.743661,0.759765,0.794766,0.807725,0.797763,0.803377
"""ARB""","""SEI""","""lr""",72,1.86,0.05,"""reverse_dynamic""",0.63,0.25,1764509495,1764517135,454.8,711.0,0.2194,0.2132,0.1402,0.1389,1,0.395187,-3.016506,0.72586,-2.290647,2,0.057939,0.056282,0.671782,1.156725,0.306154,0.17413,1.136886,0,1.352688,0.457229,0.005196,0.005496,0.455887,0.003865,0.005952,-0.293554,1.062419,1.814991,0.431642,0.410802,35.831269,32.447626,67.608092,72.668461,66.364879,70.843242,0.220001,0.219789,0.220204,0.217578,0.217343,0.217803,0.155604,-1.609113,4.470938,3.72418,0.720566,0.695017,0.834593,0.729172,0.809969,0.746511
"""ARKM""","""OP""","""dist""",72,1.67,0.37,"""reverse_dynamic""",0.13,0.8,1764294980,1764547665,400.0,300.8,0.2495,0.2269,0.3317,0.3123,1,0.384275,-9.23056,5.641805,-3.588755,2,0.060285,0.055745,1.242414,0.757202,0.097706,0.117098,1.027529,0,0.75853,-0.266194,0.006781,0.006255,-0.280387,0.008015,0.006228,5.331597,1.445266,0.638369,0.549371,0.613334,32.813329,42.776288,27.189824,31.927855,43.047193,48.26277,0.535917,0.536771,0.535098,0.582752,0.587036,0.578641,11.124957,-6.91984,16.757679,3.719429,0.719334,0.714437,0.769608,0.785846,0.813166,0.807732
"""MANTA""","""SEI""","""lr""",72,2.17,0.27,"""reverse_static""",0.23,0.21,1764298570,1764342590,873.0,723.0,0.11431,0.11638,0.138,0.1409,1,0.403037,1.605718,-2.298345,-0.692627,1,0.063936,0.056282,1.043621,0.598118,0.099074,0.024535,0.93539,0,0.680997,-0.149652,0.007067,0.00664,-0.180236,0.007049,0.004871,20.436772,4.967317,0.789924,0.520035,0.448107,38.463142,34.345616,38.474635,45.293015,31.193693,42.067948,1.152,1.150416,1.153575,1.135896,1.131507,1.140259,63.350998,-0.289363,21.533674,7.319455,0.637376,0.695017,0.714798,0.729172,0.74397,0.746511
"""ONDO""","""SEI""","""lr""",96,2.67,0.15,"""direct""",0.0,0.0,1764181100,1764245775,193.0,710.0,0.5161,0.5173,0.1404,0.1386,1,0.3975362,0.032154,1.07991,1.112064,1,0.043679,0.056282,0.552667,1.528399,0.012541,0.002413,1.689171,0,1.985598,1.285565,0.006431,0.00688,1.307192,0.008696,0.010965,1.682287,0.537959,1.208914,0.639277,0.541373,32.166572,46.282031,75.249977,77.070332,69.26765,67.454199,0.852225,0.883654,0.822418,0.541525,0.187819,0.876965,4.104229,0.751386,11.463232,4.849559,0.816739,0.695017,0.83817,0.729172,0.859071,0.746511


In [ ]:
df.sample(3)

In [ ]:
X, y = df.drop('pr_rat_test'), df['pr_rat_test']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
X_test.with_columns(
    pl.Series(y_test).alias('pr_rat_real')
).filter((pl.col('pr_rat_best') > 0.5) & (pl.col('pr_rat_all') > 0.5))#['pr_rat_real'].sum()

In [ ]:
4.753

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train);
lr_preds = lr.predict(X_test)
root_mean_squared_error(y_test.to_numpy(), lr_preds)

In [ ]:
# 0.8532 -> 0.8286 (добавил фичу std)

In [ ]:
for name, coef in zip(lr.feature_names_in_, lr.coef_):
    print(f'{name:>15}: {coef:>7.4f}')

In [ ]:
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [ ]:
params = {'eps': [0.0001, 0.001, 0.01]}

lasso = LassoCV(random_state=42)
gcv = GridSearchCV(lasso, params, cv=5, n_jobs=12, verbose=0)
gcv.fit(X_train, y_train);

In [ ]:
gcv.best_params_

In [ ]:
lasso = LassoCV(random_state=42, eps=0.0001)
lasso.fit(X_train, y_train);
lasso_preds = lasso.predict(X_test)
root_mean_squared_error(y_test.to_numpy(), lasso_preds)

In [ ]:
# 0.8211

In [ ]:
for name, coef in zip(lasso.feature_names_in_, lasso.coef_):
    print(f'{name:>15}: {coef:>7.4f}')

In [ ]:
ridge = RidgeCV()
ridge.fit(X_train, y_train);
ridge_preds = ridge.predict(X_test)
root_mean_squared_error(y_test.to_numpy(), ridge_preds)

In [ ]:
# 0.8118 -> 0.8117 (+ std)

In [ ]:
for name, coef in zip(ridge.feature_names_in_, ridge.coef_):
    print(f'{name:>15}: {coef:>7.4f}')

In [ ]:
params = {'max_features': [4, 6, 8, 10, 13], 'min_samples_leaf': [1, 3, 5, 7], 'max_depth': [5, 6, 8, 10, 12, 15],
          'n_estimators': [40, 60, 80, 100, 125, 150]}

rfr = RandomForestRegressor(random_state=42)
gcv = GridSearchCV(rfr, params, cv=5, n_jobs=12, verbose=0)
gcv.fit(X_train, y_train);

In [ ]:
gcv.best_params_

In [ ]:
rf = RandomForestRegressor(random_state=42, n_estimators=80, max_depth=12, max_features=6, min_samples_leaf=5)
rf.fit(X_train, y_train);
rf_preds = rf.predict(X_test)
root_mean_squared_error(y_test.to_numpy(), rf_preds)

In [ ]:
# 0.7634 -> 0.7750 (+std)

In [ ]:
for name, coef in zip(rf.feature_names_in_, rf.feature_importances_):
    print(f'{name:>15}: {coef:>7.4f}')

In [ ]:
X_test.with_columns(
    pl.Series(rf_preds).alias('pr_rat_pred'),
    pl.Series(y_test).alias('pr_rat_real')
).filter(pl.col('pr_rat_pred') > 0.4)#['pr_rat_real'].sum()

In [ ]:
# 4.325

In [ ]:
params = {'learning_rate': [0.01, 0.02, 0.05, 0.1, 0.2], 'max_features': [4, 6, 8, 10, 13], 
          'min_samples_leaf': [1, 3, 5, 7], 'max_depth': [5, 6, 8, 10, 12, 15],
          'n_estimators': [40, 60, 80, 100, 125, 150]}

gbr = GradientBoostingRegressor(random_state=42)
gcv = GridSearchCV(gbr, params, cv=5, n_jobs=12, verbose=0)
gcv.fit(X_train, y_train);

In [ ]:
gcv.best_params_

In [ ]:
gbr = GradientBoostingRegressor(random_state=42, learning_rate=0.01, n_estimators=80, max_depth=5, 
                                max_features=4, min_samples_leaf=7)
gbr.fit(X_train, y_train);
gbr_preds = gbr.predict(X_test)
root_mean_squared_error(y_test.to_numpy(), gbr_preds)

In [ ]:
# 0.8004 -> 0.8014

In [ ]:
import catboost as cb

In [ ]:
params = {'learning_rate': [0.01, 0.03, 0.1, 0.3], 'iterations': [250, 500, 1000], 
          'depth': [6, 8, 10],
          'l2_leaf_reg': [1, 3, 5, 7]}
cbr = cb.CatBoostRegressor(random_state=42, verbose=False)
gcv = GridSearchCV(cbr, params, cv=5, verbose=0)
gcv.fit(X_train.to_numpy(), y_train.to_numpy());

In [ ]:
gcv.best_params_

In [ ]:
train_pool = cb.Pool(X_train.to_numpy(), y_train.to_numpy())

In [ ]:
param_grid = {
    'iterations': tune.randint(100, 1500),
    'learning_rate': tune.loguniform(1e-3, 0.5),
    'depth': tune.randint(4, 12),
    'l2_leaf_reg': tune.loguniform(1, 10),
}

In [ ]:
cb.__version__

In [ ]:
from catboost.utils import grid_search

In [ ]:
cbr = cb.CatBoostRegressor(random_state=42, verbose=False, learning_rate=0.01, iterations=800, depth=8,
                          loss_function='Expectile:alpha=0.7'
                          )
cbr.fit(X_train.to_numpy(), y_train.to_numpy());
cbr_preds = cbr.predict(X_test.to_numpy())
root_mean_squared_error(y_test.to_numpy(), cbr_preds)

In [ ]:
cbr.save_model('./data/catboost_model.json', format='json')

In [ ]:
# 0.7172

In [ ]:
for name, coef in zip(X_test.columns, cbr.feature_importances_):
    print(f'{name:>15}: {coef:>7.4f}')

In [ ]:
X_test.with_columns(
    pl.Series(cbr_preds).alias('pr_rat_pred'),
    pl.Series(y_test).alias('pr_rat_real')
).filter(pl.col('pr_rat_pred') > 0.4)['pr_rat_real'].sum()

In [ ]:
# RMSE: 5.73
# Quantile, alpha=0.7 : 9.45,  RMSE: 0.7362
# Quantile, alpha=0.65: 9.07,  RMSE: 0.7172
# Quantile, alpha=0.6 : 10.04, RMSE: 0.7522
# Quantile, alpha=0.55: 6.65,  RMSE: 0.7392
# Quantile, alpha=0.5 : 5.06, RMSE: 0.7650
# Quantile, alpha=0.4 : 3.56, RMSE: 0.7893
# Quantile, alpha=0.3 : 3.56,  RMSE: 0.8197

# Expectile, alpha=0.3 : 6.25, RMSE: 0.7571
# Expectile, alpha=0.4 : 8.97, RMSE: 0.7603
# Expectile, alpha=0.5 : 9.60, RMSE: 0.7435
# Expectile, alpha=0.6 : 8.97, RMSE: 0.7535
# Expectile, alpha=0.65: 9.60, RMSE: 0.7323
# Expectile, alpha=0.7 : 9.60, RMSE: 0.7214
# Expectile, alpha=0.75: 10.04, RMSE: 0.7260
# Expectile, alpha=0.8 : 9.87, RMSE: 0.7344

In [ ]:
X_test.with_columns(
    pl.Series(cbr_preds).alias('pr_rat_pred'),
    pl.Series(y_test).alias('pr_rat_real')
).filter(pl.col('pr_rat_pred') > 0.4)

In [ ]:
model = cb.CatBoostRegressor()
model.load_model('./data/catboost_model.json', format='json')

In [ ]:
df.sample(2)

In [ ]:
df = df.drop('coin1', 'coin2', 'dur_best', 
            # 'dur_test', 'trades_test', 'loss_test', 'max_drdn_test', 'pr_test'
            )

X = df.to_numpy()

In [ ]:
preds = model.predict(X)

In [ ]:
df = pairs.select('coin1', 'coin2', 'dist', 'std', 'corr', 'pv_1', 'pv_2', 'pr_ind', 'pr_rat_ind', 'avg_pr_all', 'pr_rat_all',
            'pr_best', 'pr_rat_best', 'loss_best', 'max_drdn_best', 'dur_best',
            # 'pr_test', 'pr_rat_test', 'loss_test', 'max_drdn_test', 'dur_test', 'trades_test'
                 )

trade_pairs_df = df.with_columns(
        pl.Series(preds).alias('pred'),
    ).filter(
        (pl.col('pr_rat_ind') > 0.4) & (pl.col('pr_rat_all') > 0.4) & (pl.col('pr_rat_best') > 0.4) #& (pl.col('pred') > 0.4)
    ).sort(by='pr_rat_all', descending=True)


In [ ]:
trade_pairs_df

In [ ]:
used_tokens = []
trade_pairs_list = []

for row in trade_pairs_df.iter_rows(named=True):
    t1 = row['coin1']
    t2 = row['coin2']
    
    if t1 in used_tokens or t2 in used_tokens:
        continue

    trade_pairs_list.append((t1, t2))
    used_tokens.append(t1)
    used_tokens.append(t2)

In [ ]:
used_tokens

In [ ]:
trade_pairs_list

In [ ]:
from jaref_bot.db.postgres_manager import DBManager
from jaref_bot.config.credentials import host, user, password, db_name

db_params = {'host': host, 'user': user, 'password': password, 'dbname': db_name}
db_manager = DBManager(db_params)

In [ ]:
current_pairs = db_manager.get_table('pairs', df_type='polars')
current_pairs

In [ ]:
for row in current_pairs.iter_rows(named=True):
    t1 = row['token_1'][:-5]
    t2 = row['token_2'][:-5]
    
    if (t1, t2) not in trade_pairs_list:
        trade_pairs_list.append((t1, t2))
        print((t1, t2))

In [ ]:
with open('./jaref_bot/config/token_pairs.txt', 'w') as file:
    for pair in trade_pairs_list:
        file.write(f"{pair[0]} {pair[1]}\n")